# Creating the Training Pipeline

## Imports

In [24]:
from importlib.metadata import version
import pandas as pd
import numpy as np
from pathlib import Path
import os
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import torch
from torch.nn import Module # For type hinting
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import time
import argparse
from tqdm import tqdm


In [2]:
from importlib.metadata import version

In [13]:
list = ['pandas', 'numpy', 'seaborn', 'matplotlib', 'torch', 'joblib']
for package in list:
    try:
        print(f"{package} version: {version(package)}") # Raises PackageNotFoundError if not found
    except:
         print(f"❌ Package '{package}' not found. Please install it.")

pandas version: 2.3.1
numpy version: 1.23.5
seaborn version: 0.13.2
matplotlib version: 3.10.5
torch version: 2.5.1
joblib version: 1.5.1


## Data Pipeline

### Creating Dataset Class

In [4]:
class OilDataset(Dataset):
    """Dataset class For the OIL_DATASET"""
    def __init__(self, csv_file="../Data/DataSplits/test.csv"):
        try:
            self.data = pd.read_csv(csv_file)   # Assign a pandas data frame
            
        except FileNotFoundError:
            raise FileNotFoundError(f"File not found: {csv_file}")

        # Define feature and label columns
        self.label_column = "Close"
        # Remove the Date column and the label column
        self.feature_columns = self.data.columns.drop([self.label_column])
        

    def __getitem__(self, index):
        features = self.data.loc[index, self.feature_columns].values
        
        label = self.data.loc[index, self.label_column] # Extract the label for the given index
        return (
            torch.tensor(features, dtype=torch.float),
            torch.tensor(label, dtype=torch.float)
        )

    def __len__(self):
        return len(self.data)

### Data Pipeline Function

In [28]:
def data_pipeline(root_data_dir: str= "../Data", data_file_path: str="OIL_DATASET.csv", data_splits_dir: str="DataSplits", scaler_dir = "Scalers", batch_size: int=64, num_workers=0, pin_memory: bool=False, drop_last: bool=True) -> tuple[Dataset, Dataset, Dataset, DataLoader, DataLoader, DataLoader, MinMaxScaler, MinMaxScaler]:
    """This function prepares the train, test, and validation datasets.
    Args:
        root_data_dir (str): The root of the Data Directory
        data_file_path (str): The name of the original dataset (with .csv file extension).
        data_splits_dir (str): Path to the train, test, and validation datasets.
        scaler_dir (str): Path to the feature and label scalers.
        batch_size (int): The dataloader's batch_size.
        num_workers (int): The dataloader's number of workers.
        pin_memory (bool): The dataloader's pin memory option.
        drop_last (bool): The dataloader's drop_last option.

    Returns: 
        train_dataset (Dataset): Dataset Class for the training dataset.
        test_dataset (Dataset): Dataset Class for the test dataset.
        validation_dataset (Dataset): Dataset Class for the validation dataset.
        train_dataloader (DataLoader): The train dataloader.
        test_dataloader (DataLoader): The test dataloader.
        validation_dataloader (DataLoader): The validation dataloader.
        feature_scaler (MinMaxScaler): The scaler used to scale the features of the model input.
        label_scaler (MinMaxScaler): The scaler used to scale the labels of the model input.
        """
    if not root_data_dir or not data_file_path or not data_splits_dir:  # Check for empty strings at the beginning
        raise ValueError("File and directory paths cannot be empty strings.")
    DATA_ROOT = Path(root_data_dir)
    # OIL_PATH_ORIGINAL = DATA_ROOT / "OIL_Dataset_1984-2025.csv"     # Set the data source path

    DATA_CLEAN_PATH = DATA_ROOT / data_file_path # Set the path to the complete dataset

    if DATA_CLEAN_PATH.exists():
        print(f"CSV file detected, reading from '{DATA_ROOT}'")
        df = pd.read_csv(DATA_CLEAN_PATH)
    else:
        print(f"Downloading CSV file from HuggingFace")
        os.makedirs(DATA_ROOT, exist_ok=True)       # Create the Data Root Directory
        df = pd.read_csv("hf://datasets/MaxPrestige/CRUDE_OIL_PRICES/Data/OIL_DATASET.csv")  # Download and read the data into a pandas dataframe
        df.to_csv(DATA_CLEAN_PATH, index=False)     # Save the file, omitting saving the index

    DATA_SPLITS_DIR = DATA_ROOT / data_splits_dir
    SCALER_DIR = DATA_ROOT / scaler_dir

    TRAIN_DATA_PATH = DATA_SPLITS_DIR / "train.csv"
    TEST_DATA_PATH = DATA_SPLITS_DIR / "test.csv"
    VALIDATION_DATA_PATH = DATA_SPLITS_DIR / "val.csv"

    FEATURE_SCALER_PATH = SCALER_DIR / "feature-scaler.joblib"
    LABEL_SCALER_PATH = SCALER_DIR / "label-scaler.joblib"

    label_col = "Close"
    extra_dropped_cols = 'Date'

    if os.path.exists(TRAIN_DATA_PATH) and os.path.exists(TEST_DATA_PATH) and os.path.exists(VALIDATION_DATA_PATH) :
        print(f"Train, Test, and Validation csv datasets detected in '{DATA_SPLITS_DIR}.' Skipping generation and loading scaler(s)")
        try:
            feature_scaler = joblib.load(FEATURE_SCALER_PATH)
            label_scaler = joblib.load(LABEL_SCALER_PATH)
        except Exception as e:
            raise RuntimeError(f"An unexpected error occurred when loading scalers: {e}")
    else:
        print(f"Datasets not found in '{DATA_SPLITS_DIR}' or incomplete. Generating datasets...")
        # os.makedirs(MODEL_ROOT, exist_ok=True)
        os.makedirs(DATA_SPLITS_DIR, exist_ok=True)     # Create the Data Splits Parent Directory
        os.makedirs(SCALER_DIR, exist_ok=True)     # Create the Scaler Parent Directory

        feature_scaler = MinMaxScaler()
        label_scaler = MinMaxScaler()
        # Split the Dataframe into separate features and labels DataFrames
        df_features = df.drop(columns=[label_col, extra_dropped_cols], inplace=False)
        df_labels = df[[label_col]]     # Instead of returning a pandas Series using "[]", return a dataframe using the "[[]]" to get a shape with (-1,1)

        # Split into smaller DataFrames for the Train, Test, and Validation splits
        X_train, X_inter, Y_train, Y_inter = train_test_split(df_features, df_labels, test_size=0.1, random_state=42)
        X_validation, X_test, Y_validation, Y_test = train_test_split(X_inter, Y_inter, test_size=0.5, random_state=42)

        feature_scaler.fit(X_train)
        label_scaler.fit(Y_train)

        # Save the fitted scaler object
        try:
            joblib.dump(feature_scaler, FEATURE_SCALER_PATH)
            print(f"Feature scaler stored in: ({FEATURE_SCALER_PATH})")
            joblib.dump(label_scaler, LABEL_SCALER_PATH)
            print(f"Label scaler stored in: ({LABEL_SCALER_PATH})")
        except Exception as e:
            raise RuntimeError(f"An unexpected error occurred when saving  Scalers: {e}")

        # Scale the rest of the data; returns numpy arrays
        X_train_scaled = feature_scaler.transform(X_train)
        Y_train_scaled = label_scaler.transform(Y_train)
        X_validation_scaled = feature_scaler.transform(X_validation)
        Y_validation_scaled = label_scaler.transform(Y_validation)
        X_test_scaled = feature_scaler.transform(X_test)
        Y_test_scaled = label_scaler.transform(Y_test)

        print(f"Train Features Scaled Shape: {X_train_scaled.shape}")
        print(f"Train Labels Scaled Shape: {Y_test_scaled.shape}")
        print(f"validation Features Scaled Shape: {X_validation_scaled.shape}")
        print(f"validation Labels: {Y_validation_scaled.shape}")
        print(f"test Features Scaled Shape: {X_test_scaled.shape}")
        print(f"test Labels Scaled Shape: {Y_test_scaled.shape}")
        # Define the column names of the features and label
        features_names = df_features.columns
        label_name = df_labels.columns
        # Create dataframes using the scaled data
        X_train_df = pd.DataFrame(X_train_scaled, columns=features_names)
        X_test_df = pd.DataFrame(X_test_scaled, columns=features_names)
        X_validation_df = pd.DataFrame(X_validation_scaled, columns=features_names)
        Y_train_df = pd.DataFrame(Y_train_scaled, columns=label_name)
        Y_test_df = pd.DataFrame(Y_test_scaled, columns=label_name)
        Y_validation_df = pd.DataFrame(Y_validation_scaled, columns=label_name)

        # Concatenate the features and labels back into a single DataFrame for each set
        train_data_frame = pd.concat([X_train_df, Y_train_df.reset_index(drop=True)], axis=1)
        test_data_frame = pd.concat([X_test_df, Y_test_df.reset_index(drop=True)], axis=1)
        validation_data_frame = pd.concat([X_validation_df, Y_validation_df.reset_index(drop=True)], axis=1)

        # Saving the split data to csv files
        train_data_frame.to_csv(TRAIN_DATA_PATH, index=False)
        test_data_frame.to_csv(TEST_DATA_PATH, index=False)
        validation_data_frame.to_csv(VALIDATION_DATA_PATH, index=False)
    # Creating Datasets from the stored datasets
    print(f"Initializing Datasets")
    train_dataset = OilDataset(TRAIN_DATA_PATH)
    test_dataset = OilDataset(TEST_DATA_PATH)
    val_dataset = OilDataset(VALIDATION_DATA_PATH)
    
    print(f"Creating DataLoaders with batch_size ({batch_size}), num_workers ({num_workers}), pin_memory ({pin_memory}). Training dataset drop_last: ({drop_last})")
    train_dataloader = DataLoader(dataset=train_dataset, batch_size=batch_size, num_workers=num_workers, pin_memory=pin_memory, drop_last=drop_last, shuffle=True)
    validation_dataloader = DataLoader(dataset=val_dataset, batch_size=batch_size, num_workers=num_workers, pin_memory=pin_memory, drop_last=drop_last, shuffle=False)
    test_dataloader = DataLoader(dataset=test_dataset, batch_size=batch_size, num_workers=num_workers, pin_memory=pin_memory, drop_last=drop_last, shuffle=False)

    print(f"Training DataLoader has ({len(train_dataloader)}) batches, Test DataLoader has ({len(test_dataloader)}) batches, Validation DataLoader has ({len(validation_dataloader)}) batches")

    return (train_dataset, test_dataset, val_dataset, train_dataloader, test_dataloader, validation_dataloader, feature_scaler, label_scaler)

In [38]:
try:
    data_pipeline(root_data_dir="../Data", data_file_path="OIL_DATASET.csv", data_splits_dir="DataSplits")
except Exception as e:
    raise RuntimeError(f"An unexpected error occurred when running the data pipeline function:{e}")

CSV file detected, reading from '..\Data'
Train, Test, and Validation csv datasets detected in '..\Data\DataSplits.' Skipping generation and loading scaler(s)
Initializing Datasets
Creating DataLoaders with batch_size (64), num_workers (0), pin_memory (False). Training dataset drop_last: (True)
Training DataLoader has (89) batches, Test DataLoader has (4) batches, Validation DataLoader has (4) batches


## Agent Architecture

### Module Layer

In [ ]:
class ModuleLayer(torch.nn.Module):
    """Class for the individual layer blocks."""
    def __init__(self, intermediate_dim=32, dropout_rate=0.1):
        super().__init__()
        self.mod_linear = torch.nn.Linear(intermediate_dim, intermediate_dim)
        self.mod_norm = torch.nn.LayerNorm(normalized_shape=intermediate_dim)
        self.mod_relu = torch.nn.ReLU()
        self.dropout = torch.nn.Dropout(p=dropout_rate)

    def forward(self, x):
        x = self.mod_linear(x)
        x = self.mod_norm(x)
        x = self.mod_relu(x)
        x = self.dropout(x)
        return x

### Oil Agent

In [34]:
class Agent(torch.nn.Module):
    """Class for Agent Structure using multiple Layer Blocks."""
    def __init__(self, cfg):
        super().__init__()
        self.linear = torch.nn.Linear(in_features=cfg["in_dim"], out_features=cfg["intermediate_dim"])
        self.layers = torch.nn.Sequential(*[ModuleLayer(intermediate_dim=cfg["intermediate_dim"], dropout_rate=cfg["dropout_rate"]) for _ in range(cfg["num_blocks"])])
        self.out = torch.nn.Linear(in_features=cfg["intermediate_dim"], out_features=cfg["out_dim"])

    def forward(self, x):
        x = self.linear(x)
        x = self.layers(x)
        x = self.out(x)
        return x

## Helper Functions

### Log Iteration Functions

In [17]:
def log_iteration(batch_idx: int, total_batches: int, loss_value: float):
    """Logs the loss of the current batch."""
    print(f"Epoch batch [{batch_idx}/{total_batches}] | Loss: {loss_value:.7f}")

In [20]:
def log_epoch_iteration(epoch: int, avg_epoch_loss: float):
    """Log Current Metrics accumulated in the current epoch iteration.
    Args:
        epoch (int): the current iteration
        avg_epoch_loss (float): The average loss of the current epoch
    Returns:
        N/A
        """
    if avg_epoch_loss:
        print(f"=====================  [EPOCH ({epoch}) LOGGING]  =====================")
        print("| AVERAGES of THIS EPOCH:")
        print(f"| ACCUMULATED LOSS: {avg_epoch_loss:.7f}")
        print(f"===========================================================")
    
    else:
        print("No Data collected for this epoch to log")

### Evaluate Model Function

In [21]:
def evaluate_model(model: Module, dataloader: DataLoader, current_epoch: int = None, max_epochs: int=None, device: str = 'cpu') -> float:
    """
    Evaluates the model on a given dataset and returns the average loss.
    Args:
        model (Module): The Model.
        dataloader (DataLoader): The dataloader to calculate average loss with.
        current_epoch (int): The current epoch [optional].
        max_epochs (int): The maximum number of epochs [optional].
        device (str): The device that the calculations will take place on.
    Returns:
        avg_loss (float): The calculated average loss.
    """
    model.eval()
    total_loss = 0.0
    # loss_fn = torch.nn.MELoss(reduction='sum') # Use reduction='sum' instead of 'mean' for total loss
    loss_fn = torch.nn.L1Loss(reduction='sum')
    if len(dataloader.dataset) == 0:
        print("Warning: Evaluation dataset is empty. Skipping evaluation.")
        return float('nan')
    
    with torch.no_grad():
        for batch_inputs, batch_labels in dataloader:
            batch_inputs, batch_labels = batch_inputs.to(device), batch_labels.unsqueeze(dim=-1).to(device)
            outputs = model(batch_inputs)
            loss = loss_fn(outputs, batch_labels)
            total_loss += loss.item()
    
    avg_loss = total_loss / len(dataloader.dataset)     # Calculate the average loss on the dataset

    if current_epoch and max_epochs:   # If the function was called in the training loop
        print(f"===================  [Epoch ({current_epoch}/{max_epochs})]  ===================")
        print(f"Entire Validation Dataset Average Loss: {avg_loss:.4f}")
        print(f"====================================================")

    else:   # If the function was called outside of the training loop
        print(f"===============================================")
        print(f"Entire Dataset Average Loss: {avg_loss:.4f} ")
        print(f"=====================================================")
            
    return avg_loss

### Training Function

In [32]:
def train_model(model_config: dict, train_dataloader: DataLoader, validation_dataloader: DataLoader, model: Agent = None, epochs=32, learning_rate=0.0003, max_grad_norm=0.5, log_iterations=10, eval_iterations=10, device="cpu") -> Agent:
    """The Model Training function.

    Args:
        model_config (dict): The base configurations for building the policies.
        train_dataloader (DataLoader): The dataloader for the training loop.
        validation_dataloader (DataLoader): The dataloader for the validation loop.
        model (Agent): The model to be trained.
        epochs (int): The number of times the outer loop is performed.
        learning_rate (float): The hyperparameter that affects how much the model's parameters learn on each update iteration.
        max_grad_norm (float): Used to promote numerical stability and prevent exploding gradients.
        log_iterations (int): Used to log information about the state of the Agent.
        eval_iterations (int): Used to run an evaluation of the Agent.
        device (str): The device that the model will be trained on.

    Returns: 
        agent (Module): The Trained Model in evaluation mode.
    """
    print(f"Training Model on {device} with {epochs} main epochs, {learning_rate} learning rate, max_grad_norm={max_grad_norm}.")
    print(f"Logging every {log_iterations} epoch iterations, evaluating every {eval_iterations} epoch iterations.")

    agent = (model if model is not None else Agent(model_config)).to(device) # Create agent if nothing was passed, otherwise, create the agent. Send agent to device.

    optimizer = torch.optim.AdamW(params=agent.parameters(), lr=learning_rate, weight_decay=0.01)
    loss_fn = torch.nn.L1Loss(reduction='mean')      # Define the Loss function


    history = {'train_loss': [], 'val_loss': []}

    train_dataloader_length = len(train_dataloader)
    agent.train()   # Set agent to training mode
    for epoch in tqdm(range(epochs), desc=f">>>>>>>>>>>>>>>>>>>>>\nMain Epoch (Outer Loop)", leave=True):

        epoch_loss_total = 0.0
        for batch_idx, (inputs, labels) in enumerate(tqdm(train_dataloader, desc=f"Epoch {epoch + 1}/{epochs} - Training", leave=False)):           # Get a mini-batch of training examples from the dataloader
            # optimizer.zero_grad(set_to_none=True)       # Clear the gradients built up; Setting to None to improve performance
            optimizer.zero_grad()       # Clear the gradients built up; Setting to None to improve performance

            inputs, labels = inputs.to(device), labels.unsqueeze(dim=-1).to(device)   # Move the inputs and labels to the device

            agent_outputs = agent(inputs)       # Pass the inputs to the model and get the outputs.

            loss = loss_fn(agent_outputs, labels)      # Calculate the mini-batch loss
            epoch_loss_total += loss.item()
            
            loss.backward()         # Calculate the loss with respect to the model parameters
            torch.nn.utils.clip_grad_norm_(parameters=agent.parameters(), max_norm=max_grad_norm)   # Prevent the gradients from affecting the model parameters too much and reduce the risk of exploding gradients

            optimizer.step()      # Update the model's parameters using the learning rate

            # LOGGING LOSS OF CURRENT ITERATION
            if (batch_idx + 1) % log_iterations == 0:
                log_iteration(batch_idx=(batch_idx + 1), total_batches=train_dataloader_length, loss_value=loss.item())

        # CALCULATE AND STORE THE AVERAGE EPOCH LOSS
        epoch_avg_loss = epoch_loss_total / train_dataloader_length
        history["train_loss"].append(epoch_avg_loss)

        # LOG THE AVERAGE LOSS OF THE EPOCH
        log_epoch_iteration(epoch=epoch, avg_epoch_loss=epoch_avg_loss)

        # EVALUATE THE MODEL
        if (epoch + 1) % eval_iterations == 0:
            val_loss = evaluate_model(model=agent, dataloader=validation_dataloader, current_epoch=(epoch + 1), max_epochs=epochs, device=device)
            history["val_loss"].append(val_loss)
            agent.train()   # Set agent to training mode
        
    return agent.eval(), history

## Main Loop

In [ ]:
def main(args) -> int:
    print("SETTING UP FOR TRAINING")
    
    if args.device:     # Check if the user specified to use a CPU or GPU for training
        device = args.device
    else:
        if args.use_cuda:   # Check if the user wanted to use CUDA if available.
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    SAVE_LOCATION = "./models/Agent.pt"   # Define the model path and name of the trained model weights

    BASE_CONFIG={
    "in_dim": 22,    # Number of Features as input
    "intermediate_dim": 128,    
    "out_dim": 1,   
    "num_blocks": 12,   # Number of reapeating Layer Blocks
    "dropout_rate": 0.1     # Rate for dropout layer
}

    # --- Data Preparation Pipeline --- 
    try:
        (train_dataset, test_dataset, validation_dataset, train_dataloader, test_dataloader, validation_dataloader, feature_scaler, label_scaler) = data_pipeline(batch_size=args.dataloader_batch_size)
    except ValueError as e:
        print(f"Caught an error: {e}")

    print("BEGINNING TRAINING SCRIPT")
    start_time=time.time()

    trained_policy, training_history = train_model(
        model_config=BASE_CONFIG,
        train_dataloader=train_dataloader,
        validation_dataloader=validation_dataloader,
        model=None,     # Create new model
        epochs=args.epochs,
        learning_rate=args.learning_rate,
        max_grad_norm=args.max_grad_norm,
        log_iterations=args.log_iterations,
        eval_iterations=args.eval_iterations,
        device=device,
    )
    end_time=time.time()

    # --- Calculate Training Time --- 

    elapsed_time= end_time - start_time
    hrs = int(elapsed_time / 3600)
    min = int((elapsed_time % 3600) / 60)
    seconds_remaining = elapsed_time - (hrs * 3600 ) - (min * 60)

    print(f"FINISHED MODEL TRAINING. \nTRAINING TOOK: {hrs} Hours, {min} Minutes, and {seconds_remaining:.3f} Seconds")

    # --- Testing Trained Model --- 
    print("\nTESTING THE TRAINED POLICY:")
    test_loss = evaluate_model(model=trained_policy, dataloader=test_dataloader, current_epoch=None, max_epochs=None, device='cpu')

    # ---  Saving Model Section  ---   

    if args.save_model:     # Check if the user wants to save the trained model weights
        if args.model_output_path:     # Check if the user specified a target save location
            SAVE_LOCATION=args.model_output_path

        parent_dir = os.path.dirname(SAVE_LOCATION)

        # If parent_dir is empty, it means the SAVE_LOCATION is just a filename
        # in the current directory, so no new directories need to be created.
        if parent_dir and parent_dir != '.':
            try:
                os.makedirs(parent_dir, exist_ok=True)
                print(f"Parent directory '{parent_dir}' created to store the model.")
            except OSError as e:
                print(f"Error creating directory {parent_dir}: {e}")
                SAVE_LOCATION='model.pt'      # Fall back to a default save location if problem occurs.
        
        try:
            torch.save(trained_policy.state_dict(), f=SAVE_LOCATION)
            print(f"Model weights saved in: {SAVE_LOCATION}")
        except Exception as e:
            print(f"Error saving model to {SAVE_LOCATION}: {e}")

    return 0

In [37]:
# Example usage (assuming you have a way to call this function, e.g., in a main block)
if __name__ == '__main__':
    # --- Begin Timing Main Script Execution Time ---
    main_start_time=time.time()

    parser = argparse.ArgumentParser(description="Train and evaluate a Regression Agent.")

    parser.add_argument('--epochs', type=int, default=8,
        help='(int, default=8) Number of training epochs to run.')

    parser.add_argument('--learning_rate', type=float, default=0.0003,
        help='(float, default=0.0003) Learning rate used by the optimizer.')
    
    parser.add_argument('--max_grad_norm', type=float, default=3.0,
        help='(float, default=3.0) The Maximum L2 Norm of the gradients for Gradient Clipping.')

    parser.add_argument('--dataloader_batch_size', type=int, default=64,
        help='(int, default=64) Batch size used by the dataloaders for training, validation, and testing.')

    parser.add_argument('--dataloader_pin_memory', action='store_false',
        help='(bool, default=True) Disable pinned memory in dataloaders (enabled by default).')

    parser.add_argument('--dataloader_num_workers', type=int, default=0,
        help='(int, default=0) Number of subprocesses to use for data loading.')

    parser.add_argument('--log_iterations', type=int, default=32,
        help='(int, default=32) Frequency (in iterations) to log training progress.')

    parser.add_argument('--eval_iterations', type=int, default=32,
        help='(int, default=32) Frequency (in iterations) to evaluate the model.')

    parser.add_argument('--use_cuda', action='store_true',
        help='(bool, default=False) Enable CUDA for training if available.')

    parser.add_argument('--device', type=str, default='cpu',
        help='(str, default="cpu") Device to use for training (e.g., "cpu", "cuda:0"). Overrides --use_cuda.')

    parser.add_argument('--save_model', action='store_true',
        help='(bool, default=False) Save the trained model after training.')

    parser.add_argument('--model_output_path', type=str, default='models/Weather-Agent.pt',
        help='(str, default="models/Weather-Agent.pt") File path to save the trained model.')


    # # Parse the arguments (disabled for ipynb testing)
    # args = parser.parse_args()

    # For ipynb testing
    simulated_args = [
        '--epochs', '256',
        '--learning_rate', '0.003',
        '--log_iterations', '256',  # Basically turns logging off
        '--eval_iterations', '64',
        '--save_model',
        '--model_output_path', 'models/Agent_01.pt'
        #Everything else is default
    ]
    args = parser.parse_args(args=simulated_args)
    print(args)
    ## End of ipynb testing

    ret = main(args)

    main_end_time=time.time()

    # --- Calculate Main Script Execution Time --- 

    elapsed_time= main_end_time - main_start_time
    hrs = int(elapsed_time / 3600)
    min = int((elapsed_time % 3600) / 60)
    seconds_remaining = elapsed_time - (hrs * 3600 ) - (min * 60)

    print(f"FINISHED MAIN SCRIPT\nOVERALL DURATION: {hrs} Hours, {min} Minutes, and {seconds_remaining:.3f} Seconds")
    if ret == 0:
        print("TERMINATING PROGRAM")
    else: 
        print("Main Scipt Error")

Namespace(epochs=256, learning_rate=0.003, max_grad_norm=3.0, dataloader_batch_size=64, dataloader_pin_memory=True, dataloader_num_workers=0, log_iterations=256, eval_iterations=64, use_cuda=False, device='cpu', save_model=True, model_output_path='models/Weather-Agent_01.pt')
SETTING UP FOR TRAINING
CSV file detected, reading from '..\Data'
Train, Test, and Validation csv datasets detected in '..\Data\DataSplits.' Skipping generation and loading scaler(s)
Initializing Datasets
Creating DataLoaders with batch_size (64), num_workers (0), pin_memory (False). Training dataset drop_last: (True)
Training DataLoader has (89) batches, Test DataLoader has (4) batches, Validation DataLoader has (4) batches
BEGINNING TRAINING SCRIPT
Training Model on cpu with 256 main epochs, 0.003 learning rate, max_grad_norm=3.0.
Logging every 256 epoch iterations, evaluating every 64 epoch iterations.


>>>>>>>>>>>>>>>>>>>>>
Main Epoch (Outer Loop):   0%|          | 1/256 [00:02<08:33,  2.01s/it]

=====================  [EPOCH (0) LOGGING]  =====================
| AVERAGES of THIS EPOCH:
| ACCUMULATED LOSS: 0.2078976


>>>>>>>>>>>>>>>>>>>>>
Main Epoch (Outer Loop):   1%|          | 2/256 [00:03<07:59,  1.89s/it]

=====================  [EPOCH (1) LOGGING]  =====================
| AVERAGES of THIS EPOCH:
| ACCUMULATED LOSS: 0.1812691


...

=====================  [EPOCH (253) LOGGING]  =====================
| AVERAGES of THIS EPOCH:
| ACCUMULATED LOSS: 0.1762169


>>>>>>>>>>>>>>>>>>>>>
Main Epoch (Outer Loop): 100%|█████████▉| 255/256 [09:22<00:02,  2.24s/it]

=====================  [EPOCH (254) LOGGING]  =====================
| AVERAGES of THIS EPOCH:
| ACCUMULATED LOSS: 0.1763119


>>>>>>>>>>>>>>>>>>>>>
Main Epoch (Outer Loop): 100%|██████████| 256/256 [09:24<00:00,  2.21s/it]

=====================  [EPOCH (255) LOGGING]  =====================
| AVERAGES of THIS EPOCH:
| ACCUMULATED LOSS: 0.1762398
===================  [Epoch (256/256)]  ===================
Entire Validation Dataset Average Loss: 0.1377
FINISHED MODEL TRAINING. 
TRAINING TOOK: 0 Hours, 9 Minutes, and 24.666 Seconds

TESTING THE TRAINED POLICY:
Entire Dataset Average Loss: 0.1392 
Parent directory 'models' created to store the model.
Model weights saved in: models/Weather-Agent_01.pt
FINISHED MAIN SCRIPT
OVERALL DURATION: 0 Hours, 9 Minutes, and 24.753 Seconds
TERMINATING PROGRAM
